**特征码41 0F BE 72 10找到 地址  大概是这种东西  13EED4A1054 - 41 0FBE 72 10         - movsx esi,byte ptr [r10+10]

In [1]:
import pymem
from pymem.pattern import pattern_scan_all


def aob_injection(process, aob):
    pm = pymem.Pymem(process)  # Open a handle to the process
    aob_address = pattern_scan_all(
        pm.process_handle, aob
    )  # Scan the entire memory of the process for the AOB
    print(aob_address)  # Print the address where the AOB was found
    print(hex(aob_address))

    return aob_address


#aob = bytes.fromhex("41 0F BE 71 10 45")  # Define the AOB r9+10
aob = bytes.fromhex("41 0F BE 72 10 45 0F")  # Define the AOB r10+10
aob_address=aob_injection(
    "javaw.exe", aob
)  # Call the function with the name of the process and the AOB
#打印出来的地址需要变成jmp

2476528386132
0x2409c984054


开辟空间 叫他TR

In [2]:
pm = pymem.Pymem("javaw.exe")
TR = pymem.memory.allocate_memory(pm.process_handle, 4)
print(TR)
print(hex(TR))

1907005259776
0x1bc025f0000


In [3]:
def format_address(address, length=8):
    reversed_bytes = address.to_bytes(length, byteorder='little')
    reversed_formatted = ' '.join(f'{b:02X}' for b in reversed_bytes)
    return reversed_formatted

#print(format_address(0x1CA99861000))
TR_address_reversed_formatted=format_address(TR)
print(TR_address_reversed_formatted)


00 00 5F 02 BC 01 00 00


In [4]:
push_rax=['50']
lea_eax_rx_x=['41','8D','41','10']# r9+10
#lea_eax_rx_x=['41','8D','42','10']# r10+10
move_TR_eax=['A3']+TR_address_reversed_formatted.split(' ')# [TR],EAX
pop_rax=['58']
#movsx_esi_byte_ptr_rx_x=['41','0F' ,'BE', '71', '10']
movsx_esi_byte_ptr_rx_x=['41','0F' ,'BE', '72', '10']


In [213]:
len('50 41 8D 41 10 A3 00 10 86 99 CA 01 00 00 58 41 0F BE 71 10 E9 40 52 AC OB'.split(' '))
#'50 41 8D 41 10 A3 00 10 86 99 CA 01 00 00 58 41 0F BE 71 10 E9 40 52 AC OB'.split(' ')

25

In [5]:
part_head_combine= push_rax+lea_eax_rx_x+move_TR_eax+pop_rax+movsx_esi_byte_ptr_rx_x
part_head_combine
print('len part_head_combine',len(part_head_combine))
print(part_head_combine)
jmp_place=['E9']

len part_head_combine 20
['50', '41', '8D', '41', '10', 'A3', '00', '00', '5F', '02', 'BC', '01', '00', '00', '58', '41', '0F', 'BE', '72', '10']


In [6]:
shell_code_len=len(part_head_combine)+1+4
print('shell_code_len',shell_code_len)

shell_code_len 25


In [7]:
# Allocate new memory
newmem = pymem.memory.allocate_memory(pm.process_handle, shell_code_len)

In [8]:
print(newmem)
print(hex(newmem))

1907004276736
0x1bc02500000


In [9]:
newmen_end_row_address_jmp=hex(len(part_head_combine)+newmem)
newmen_end_row_address_jmp

'0x1bc02500014'

In [10]:
import struct

def calculate_jmp_offset(jmp_addr, target_addr):
    offset = target_addr - (jmp_addr + 5)
    return offset

def to_signed_32_bit_le(i):
    return struct.pack('<i', i)

In [11]:
# 先算 newmen中jmp 的偏移量
jmp_addr_1=int(newmen_end_row_address_jmp,16)  #多数以14结尾
print('jmp_addr_1',jmp_addr_1)
target_addr_1=aob_address+5
print('target_addr_1',target_addr_1)
print(hex(target_addr_1))
print('offset_1',jmp_addr_1, target_addr_1)
offset_1 = calculate_jmp_offset(jmp_addr_1, target_addr_1)
print('offset_1',offset_1)
offset_1_le = to_signed_32_bit_le(offset_1)
print('offset_1_le',offset_1_le)

# offset_2_le 是从 aob_address 跳到 newmen 的偏移量 A开头
jmp_addr_2=aob_address  # aob address
print('jmp_addr_2',jmp_addr_2,hex(jmp_addr_2))
target_addr_2=newmem
print('target_addr_2',target_addr_2,hex(target_addr_2))
offset_2 = calculate_jmp_offset(jmp_addr_2, target_addr_2)
# Convert to signed 32-bit little endian format
offset_2_le = to_signed_32_bit_le(offset_2)
# Print the offset in hexadecimal
print(offset_2_le.hex())


jmp_addr_1 1907004276756
target_addr_1 1908053298265
0x1bc40d6cc59
offset_1 1907004276756 1908053298265
offset_1 1049021504
offset_1_le b'@\xcc\x86>'
jmp_addr_2 1908053298260 0x1bc40d6cc54
target_addr_2 1907004276736 0x1bc02500000
a73379c1


In [12]:
hex_values=part_head_combine+jmp_place+[offset_1_le[0:1],offset_1_le[1:2],offset_1_le[2:3],offset_1_le[3:4]]
hex_values

['50',
 '41',
 '8D',
 '41',
 '10',
 'A3',
 '00',
 '00',
 '5F',
 '02',
 'BC',
 '01',
 '00',
 '00',
 '58',
 '41',
 '0F',
 'BE',
 '72',
 '10',
 'E9',
 b'@',
 b'\xcc',
 b'\x86',
 b'>']

In [13]:
#hex_string = "50 41 8D 41 10 A3 00 00 10 88 CA 01 00 00 58 41 0F BE 71 10 E9 40 52 AC 0B"
#hex_values = hex_string.split()  # split the string into a list of two-character strings
hex_values=part_head_combine+jmp_place+[offset_1_le[0:1],offset_1_le[1:2],offset_1_le[2:3],offset_1_le[3:4]]
byte_values = []
for hv in hex_values:
    if isinstance(hv, str):  # if hv is a hex string
        byte_values.append(int(hv, 16))
    elif isinstance(hv, bytes):  # if hv is a byte object
        byte_values.append(int.from_bytes(hv, 'little'))  # or 'big' if your data is big-endian
shellcode = bytes(byte_values)
print(shellcode)


b'PA\x8dA\x10\xa3\x00\x00_\x02\xbc\x01\x00\x00XA\x0f\xber\x10\xe9@\xcc\x86>'


先要创造shellcode 然后再写入

In [14]:
hex(newmem)

'0x1bc02500000'

In [15]:
result1=pm.write_bytes(newmem, shellcode, len(shellcode))

In [19]:
#现在要去aob address jmp 到newmen
inject_hex = ['E9']+[offset_2_le[0:1],offset_2_le[1:2],offset_2_le[2:3],offset_2_le[3:4]]
print(inject_hex)
byte_values = []
for hv in inject_hex:
    if isinstance(hv, str):  # if hv is a hex string
        byte_values.append(int(hv, 16))
    elif isinstance(hv, bytes):  # if hv is a byte object
        byte_values.append(int.from_bytes(hv, 'little'))  # or 'big' if your data is big-endian
inject_hex_shellcode = bytes(byte_values)
print(inject_hex_shellcode)

['E9', b'\xa7', b'3', b'y', b'\xc1']
b'\xe9\xa73y\xc1'


In [20]:
inject_hex

['E9', b'\xa7', b'3', b'y', b'\xc1']

In [21]:
print('aob_address',aob_address,hex(aob_address))

aob_address 1908053298260 0x1bc40d6cc54


In [22]:
len(inject_hex_shellcode)
print(inject_hex_shellcode)

b'\xe9\xa73y\xc1'


In [23]:
inject_hex_shellcode

b'\xe9\xa73y\xc1'

In [230]:
print(aob_address, inject_hex_shellcode, len(inject_hex_shellcode))

1996898335956 b"\xe9'\x83s\x9b" 5


In [24]:
result2=pm.write_bytes(aob_address, inject_hex_shellcode, len(inject_hex_shellcode))
print(result2)

None


In [232]:
hex(TR)

'0x1d08bdf0000'

In [2]:
import pymem

# Initialize pymem
pm = pymem.Pymem('javaw.exe')  # Replace 'YourProcessName.exe' with the name of your process

# Define the memory address


# Read bytes from the memory
# Assuming you want to read 4 bytes (an integer) at that address
data = pm.read_bytes(2940343222272, 4)
print(data)

# Convert the bytes to an integer
value = int.from_bytes(data, byteorder='little')  # Use 'big' if the byteorder is big endian

print(value)


b'\xa8/C\xf4'
4098043816


In [3]:
data = pm.read_bytes(value-4, 10)

In [4]:
data

b'\x06\x00\t\x00\x012\x0b\x00\x02\x01'

In [5]:
# Convert the first 2 bytes to an integer
value = int.from_bytes(data[:2], byteorder='little')  # Use 'big' if the byteorder is big endian

print(value)  # Output: 12081

6


In [6]:
# Convert the first 2 bytes to an integer
value = int.from_bytes(data[2:4], byteorder='little')  # Use 'big' if the byteorder is big endian

print(value)  # Output: 12081

9


In [7]:
value = int.from_bytes(data[4:6], byteorder='little')  # Use 'big' if the byteorder is big endian
print(value)  # Output: 12081

12801


In [8]:
value = int.from_bytes(data[8:], byteorder='little')  # Use 'big' if the byteorder is big endian
print(value)  # Output: 770

258


In [12]:
hex_value = hex(data[9])
#hex_value[-3] #3 right,0 down
hex_value[-1]

'1'